In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import mlflow
import pandas as pd
import numpy as np
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

from lib.reproduction import major_oxides



In [3]:
from lib import full_flow_dataloader
from lib.cross_validation import custom_kfold_cross_validation_new


# Load the data and create folds
def get_data_indices(target: str):
    # Simulate loading the data
    train_full, test_full = full_flow_dataloader.load_full_flow_data(load_cache_if_exits=True, average_shots=True)
    full_data = pd.concat([train_full, test_full], axis=0)

    # Generate custom folds
    folds, train, test = custom_kfold_cross_validation_new(
        data=full_data, k=5, group_by="Sample Name", target=target, random_state=42
    )

    train_indices = train.index.values
    index_mapping = {idx: pos for pos, idx in enumerate(train_indices)}

    # Convert DataFrame-based folds to index-based folds
    fold_indices = []
    for train_fold, test_fold in folds:
        train_fold_idx = [index_mapping[idx] for idx in train_fold.index.values if idx in index_mapping]
        test_fold_idx = [index_mapping[idx] for idx in test_fold.index.values if idx in index_mapping]
        fold_indices.append((train_fold_idx, test_fold_idx))

    return fold_indices, train, test, full_data, full_data[target]

In [4]:
target = "SiO2"
assert target in major_oxides, f"Target {target} not in major_oxides"
drop_cols = major_oxides + ["ID", "Sample Name"]

fold_indices, train_data, test_data, full_data, y_full = get_data_indices(target=target)
X_full = full_data.drop(columns=drop_cols)

In [5]:
# Manually perform cross-validation to generate predictions from base estimators
def perform_manual_cv(fold_indices, X_full, y_full, base_estimators):
    meta_features = np.zeros((X_full.shape[0], len(base_estimators)))
    for i, (train_idx, test_idx) in enumerate(fold_indices):
        X_train, X_test = X_full.iloc[train_idx], X_full.iloc[test_idx]
        y_train, y_test = y_full.iloc[train_idx], y_full.iloc[test_idx]

        for j, (name, estimator) in enumerate(base_estimators):
            estimator.fit(X_train, y_train)
            meta_features[test_idx, j] = estimator.predict(X_test)

    return meta_features


# meta_features = perform_manual_cv(
#     fold_indices, X_full, y_full, sio2_pipeline
# )

In [6]:
# Perform cross-validation for the final estimator using the meta-features
def perform_final_cv(fold_indices, meta_features, y_full, final_estimator, metric_fns):
    cv_metrics = []
    for train_idx, test_idx in fold_indices:
        X_train, X_test = meta_features[train_idx], meta_features[test_idx]
        y_train, y_test = y_full.iloc[train_idx], y_full.iloc[test_idx]

        final_estimator.fit(X_train, y_train)
        y_pred = final_estimator.predict(X_test)

        fold_metrics = [metric_fn(y_test, y_pred) for metric_fn in metric_fns]
        cv_metrics.append(fold_metrics)

    return cv_metrics

In [7]:
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR

final_estimator = SVR()

In [8]:
from lib.metrics import rmse_metric, std_dev_metric

metric_fns = [rmse_metric, std_dev_metric]

In [9]:
from experiments.stacking_pipelines import make_oxide_pipelines

oxide_pipelines = make_oxide_pipelines()

In [10]:
def do_cv_metrics(cv_metrics):
    rmse_cv_scores = [metrics[0] for metrics in cv_metrics]
    rmse_cv_mean = np.mean(rmse_cv_scores)
    rmse_cv_std = np.std(rmse_cv_scores)
    print(f"RMSECV: {rmse_cv_mean} ± {rmse_cv_std}")

    std_dev_cv_scores = [metrics[1] for metrics in cv_metrics]
    std_dev_cv_mean = np.mean(std_dev_cv_scores)
    std_dev_cv_std = np.std(std_dev_cv_scores)
    print(f"STDCV: {std_dev_cv_mean} ± {std_dev_cv_std}")

    return {
        "rmse_cv_mean": rmse_cv_mean,
        "rmse_cv_std": rmse_cv_std,
        "std_dev_cv_mean": std_dev_cv_mean,
        "std_dev_cv_std": std_dev_cv_std,
    }

In [11]:
from experiments.stacker import Stacker

def run(target: str):
    fold_indices, train_data, test_data, full_data, y_full = get_data_indices(target=target)
    X_full = full_data.drop(columns=drop_cols)
    pipeline = oxide_pipelines[target]

    stacker = Stacker(base_estimators=pipeline, final_estimator=final_estimator)
    stacker.fit_cv(X_full, y_full, fold_indices)
    meta_features = stacker.meta_features_
    cv_metrics = stacker.cv(fold_indices, meta_features, y_full, final_estimator, metric_fns)
    metrics = do_cv_metrics(cv_metrics)

    X_train_final = train_data.drop(columns=drop_cols)
    y_train_final = train_data[target]
    X_test_final = test_data.drop(columns=drop_cols)
    y_test_final = test_data[target]

    stacker.fit(X_train_final, y_train_final)
    y_pred_final = stacker.predict(X_test_final)
    rmsep_final = rmse_metric(y_test_final, y_pred_final)
    print(f"RMSEP: {rmsep_final}")

    metrics["RMSEP"] = rmsep_final

    return metrics

In [14]:
oxide_pipelines["TiO2"]

[('svr',
  Pipeline(steps=[('preprocessor',
                   Pipeline(steps=[('scaler', Norm3Scaler()),
                                   ('transformer', PowerTransformer())])),
                  ('svr',
                   SVR(C=0.0092802848242038, coef0=8.63601100525176,
                       epsilon=0.0028037787477313, kernel='poly',
                       max_iter=20000000))])),
 ('rf',
  Pipeline(steps=[('preprocessor',
                   Pipeline(steps=[('scaler', Norm3Scaler()),
                                   ('transformer',
                                    QuantileTransformer(n_quantiles=941,
                                                        random_state=42,
                                                        subsample=60746))])),
                  ('rf',
                   RandomForestRegressor(max_depth=15, max_features='sqrt',
                                         min_samples_leaf=4, n_estimators=136))]))]

In [12]:
import time

current_time = time.strftime("%Y-%m-%d %H:%M:%S")
experiment_name = f"Stacking Ensemble {current_time}"
mlflow.set_experiment(experiment_name)

# worse = ["TiO2", "Al2O3", "FeOT", "CaO", "Na2O", "K2O"]

for target in major_oxides:
    with mlflow.start_run(
        run_name=f"{target}",
        description=f"Final stacking ensemble for {target}",
    ):
        metrics = run(target)
        mlflow.log_metrics(metrics)

2024/06/11 02:21:26 INFO mlflow.tracking.fluent: Experiment with name 'Stacking Ensemble 2024-06-11 02:21:26' does not exist. Creating a new experiment.


RMSECV: 4.924651778686846 ± 0.5102940450743484
STDCV: 4.899160516250246 ± 0.5019635907932811
RMSEP: 3.398259226544109
RMSECV: 0.3853338313879524 ± 0.09195307342774808
STDCV: 0.38391664439049955 ± 0.09182382206434082
RMSEP: 0.3414117035834202
RMSECV: 2.2270189687252517 ± 0.23942256612809215
STDCV: 2.1937176174876423 ± 0.25984731160605357
RMSEP: 1.8097055061022407
RMSECV: 4.443198463846926 ± 1.7995960245062697
STDCV: 4.403320287010048 ± 1.7468337047632505
RMSEP: 1.5992961244631787
RMSECV: 2.578317043422149 ± 1.7999830810590802
STDCV: 2.5614757844189784 ± 1.777085212515416
RMSEP: 0.8278109128782456
RMSECV: 1.4033956712640174 ± 0.1414885527756463
STDCV: 1.3847443034040967 ± 0.12999286156266857
RMSEP: 1.6012293309063954
RMSECV: 0.9290148438674636 ± 0.8745584415958435
STDCV: 0.9224132062229127 ± 0.8710165671031107
RMSEP: 0.3479270874033303
RMSECV: 0.6722920697552249 ± 0.14785783463743077
STDCV: 0.6694300230946136 ± 0.14997717420963638
RMSEP: 0.4370848960261696


In [16]:
from itertools import combinations
from experiments.stacker import Stacker
from experiments.stacking_pipelines import tio2_instantiate_all_models
import mlflow

oxide = "TiO2"
experiment_name = f"{oxide} Combination Optimization - Stacker"
try:
    experiment_id = mlflow.create_experiment(experiment_name)
except mlflow.exceptions.MlflowException as e:
    if "RESOURCE_ALREADY_EXISTS" in str(e):
        experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id
    else:
        raise e


def generate_combinations(models, max_len):
    """Generate all combinations of the models up to the given length."""
    all_combinations = []
    for i in range(1, max_len + 1):
        comb = list(combinations(models, i))
        all_combinations.extend(comb)
    return all_combinations


tio2_all_models = tio2_instantiate_all_models()


def run(target: str, model_combinations, final_estimator, metric_fns, drop_cols):
    best_metrics = None
    best_combination = None

    for combo in model_combinations:
        print(f"Evaluating combination: {combo}")

        # Prepare the pipeline with the current combination of base estimators
        pipeline = [(model, tio2_all_models[model]) for model in combo]

        fold_indices, train_data, test_data, full_data, y_full = get_data_indices(target=target)
        X_full = full_data.drop(columns=drop_cols)

        stacker = Stacker(base_estimators=pipeline, final_estimator=ElasticNet())
        stacker.fit_cv(X_full, y_full, fold_indices)
        meta_features = stacker.meta_features_
        cv_metrics = stacker.cv(fold_indices, meta_features, y_full, final_estimator, metric_fns)
        metrics = do_cv_metrics(cv_metrics)

        X_train_final = train_data.drop(columns=drop_cols)
        y_train_final = train_data[target]
        X_test_final = test_data.drop(columns=drop_cols)
        y_test_final = test_data[target]

        stacker.fit(X_train_final, y_train_final)
        y_pred_final = stacker.predict(X_test_final)
        rmsep_final = rmse_metric(y_test_final, y_pred_final)
        print(f"RMSEP: {rmsep_final}")

        metrics["RMSEP"] = rmsep_final

        # Log metrics to mlflow
        with mlflow.start_run(experiment_id=experiment_id, run_name=f"Combination: {combo}"):
            mlflow.log_params({"base_estimators": combo, "final_estimator": final_estimator.__class__.__name__})
            mlflow.log_metrics(metrics)

        # Keep track of the best combination
        if best_metrics is None or metrics["RMSEP"] < best_metrics["RMSEP"]:
            best_metrics = metrics
            best_combination = combo

    print(f"Best combination: {best_combination} with RMSEP: {best_metrics['RMSEP']}")
    return best_metrics


# Define the models and final estimator
top_models = ["svr", "gbr", "xgboost", "rf", "elasticnet", "extra_trees", "ridge", "lasso", "ngboost", "pls"]
final_estimator = ElasticNet()
metric_fns = [rmse_metric, std_dev_metric]
drop_cols = major_oxides + ["ID", "Sample Name"]

# Generate all possible combinations of the top models
model_combinations = generate_combinations(top_models, max_len=3)

# Run the grid search for the best stacking ensemble configuration
results = run(
    target=oxide,
    model_combinations=model_combinations,
    final_estimator=final_estimator,
    metric_fns=metric_fns,
    drop_cols=drop_cols,
)

Evaluating combination: ('svr',)


RMSECV: 0.7999238124212151 ± 0.07241006386998335
STDCV: 0.7989029470639817 ± 0.07254436308898864
RMSEP: 0.5723841791623554
Evaluating combination: ('gbr',)
RMSECV: 0.8014940014402825 ± 0.07048436772617288
STDCV: 0.8004700533855786 ± 0.0706524967815489
RMSEP: 0.5723841791623554
Evaluating combination: ('xgboost',)
RMSECV: 0.8003374311546776 ± 0.07189782110933146
STDCV: 0.7993171355214823 ± 0.07203922763482848
RMSEP: 0.5723841791623554
Evaluating combination: ('rf',)
RMSECV: 0.8014940014402825 ± 0.07048436772617288
STDCV: 0.8004700533855786 ± 0.0706524967815489
RMSEP: 0.5723841791623554
Evaluating combination: ('elasticnet',)
RMSECV: 0.7990565231359257 ± 0.07349523372140125
STDCV: 0.7980391912922657 ± 0.07360912374575634
RMSEP: 0.5723841791623554
Evaluating combination: ('extra_trees',)
RMSECV: 0.8014940014402825 ± 0.07048436772617288
STDCV: 0.8004700533855786 ± 0.0706524967815489
RMSEP: 0.5723841791623554
Evaluating combination: ('ridge',)
RMSECV: 0.8005815199707378 ± 0.0715971751823228